In [ ]:
 !pip3 install pulp

 !pip install k-means-constrained

import pandas as pd
import pulp
import numpy as np
import random
import datetime
import matplotlib.pyplot as plt 
from sklearn.cluster import KMeans

import folium
from collections import namedtuple
from k_means_constrained import KMeansConstrained

     |████████████████████████████████| 14.2 MB 25.7 MB/s 
     |████████████████████████████████| 561 kB 24.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 15.5 MB 51.3 MB/s 
     |████████████████████████████████| 38.1 MB 1.3 MB/s 
     |████████████████████████████████| 1.0 MB 64.4 MB/s 
  Created wheel for k-means-constrained: filename=k_means_constrained-0.7.0-cp37-cp37m-linux_x86_64.whl size=1408478 sha256=4638ff1b6d8bbec4a543dbce4168793792236df9100cd45e694d2addd2807967
  Stored in directory: /root/.cache/pip/wheels/97/2e/70/c473dfd41bc709dd3c2a89ae09d345a5ffb77d1906a2a77176
Successfully built k-means-constrained
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.17.3
    Uninstalling protobuf-3.17.3:
      Successfully uninstalled protobuf-3.17.3
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Unins

# Single Driver per Cluster

In [ ]:
df_distance = pd.read_csv("distances.csv")
df_drivers4 = pd.read_csv("part4_drivers.csv")
df_4 = pd.read_csv("part4_large.csv")
#df_4 = pd.read_csv("part3_small.csv")
regions = pd.read_csv("regions.csv")

start_time = pd.to_datetime('2022-04-02 17:00:00')
W = 120

In [ ]:
occur1 = df_4['restaurant'].value_counts()
repeatsRest = occur1[occur1 > 1]

occur2 = df_4['customer'].value_counts()
repeatsCust = occur2[occur2 > 1]

In [ ]:
df_4_rest = pd.DataFrame(df_4['restaurant'])

df_4_rest = df_4_rest.rename(columns={"restaurant": "name"})

df_4_cust = pd.DataFrame(df_4[['customer']])
df_4_cust = df_4_cust.rename(columns={"customer": "name"})

In [ ]:
for repeat in repeatsRest.index:
    print(repeat)
    test = df_4.index[df_4['restaurant'] == repeat].tolist()

    p = 0
    for index in test:
        df_4.loc[df_4.index == index,  'restaurant'] = repeat + '_' + str(p)
        p += 1

for repeat in repeatsCust.index:
    print(repeat)
    test = df_4.index[df_4['customer'] == repeat].tolist()

    p = 0
    for index in test:
        df_4.loc[df_4.index == index,  'customer'] = repeat + '_' + str(p)
        p += 1

Downtown Toronto (St. James Park)
Downtown Toronto (Rosedale)
Downtown Toronto (Central Bay Street)


In [ ]:
regions = regions[['name','latitude','longitude']]

In [ ]:
merged_cust = df_4_cust.merge(regions, on = 'name', how = 'left')
merged_rest = df_4_rest.merge(regions, on = 'name', how = 'left')

merged_cust = merged_cust.rename(columns={"latitude": "lat_cust", "longitude": "long_cust"})
merged_rest = merged_rest.rename(columns={"latitude": "lat_rest", "longitude": "long_rest"})

In [ ]:
merged = pd.concat([merged_rest, merged_cust], axis=1)

In [ ]:
merged

,name,lat_rest,long_rest,name,lat_cust,long_cust
0,Downtown Toronto (Central Bay Street),43.6564,-79.3860,North York (Armour Heights / Wilson Heights / ...,43.7535,-79.4472
1,Downtown Toronto (Christie),43.6683,-79.4205,West Toronto (Brockton / Parkdale Village / Ex...,43.6383,-79.4301
2,Downtown Toronto (Ryerson),43.6572,-79.3783,York (Fairbank / Oakwood),43.6889,-79.4507
3,Downtown Toronto (St. James Park),43.6513,-79.3756,East Toronto (The Beaches),43.6784,-79.2941
4,Downtown Toronto (Kensington Market / Chinatow...,43.6541,-79.3978,Downtown Toronto (Central Bay Street),43.6564,-79.3860
5,Downtown Toronto (Rosedale),43.6827,-79.3730,Downtown Toronto (CN Tower / King and Spadina ...,43.6404,-79.3995
6,Downtown Toronto (Regent Park / Port of Toronto),43.6555,-79.3626,Downtown Toronto (Toronto Dominion Centre / De...,43.6469,-79.3823
7,Downtown Toronto (St. James Town / Cabbagetown),43.6684,-79.3689,York (Runnymede / The Junction North),43.6748,-79.4839
8,Downtown Toronto (Underground city),43.6492,-79.3823,Central Toronto (Summerhill West / Rathnelly /...,43.6861,-79.4025
9,Downtown Toronto (Church and Wellesley),43.6656,-79.3830,East Toronto (The Danforth West / Riverdale),43.6803,-79.3538


In [ ]:
centroid_Lat = merged['lat_rest'].mean()
centroid_Long = merged['long_rest'].mean()

### Define Clustering

In [ ]:
# Cluster by angle between restaurant and Customer
merged['measure'] = np.arctan((merged['long_cust'] - merged['long_rest'])/(merged['lat_cust'] - merged['lat_rest']))

# Cluster by angle between resturant centroid and customer
#merged['measure'] = np.arctan((merged['long_cust'] - centroid_Long)/(merged['lat_cust'] - centroid_Lat))

#Cluster by Radius 
#merged['measure'] = np.sqrt((merged['long_cust'] - centroid_Long)**2/(merged['lat_cust'] - centroid_Lat)**2)

In [ ]:
clusters = 3

clf = KMeansConstrained(
     n_clusters=clusters,
     size_min=2,
     size_max=6)

#clf = KMeansConstrained(
     #n_clusters=clusters,
     #size_min=1,
     #size_max=None)

In [ ]:
clusterList = clf.fit_predict(np.array(merged['measure']).reshape(-1, 1))
merged['cluster'] = clf.labels_
clusterList
#merged['cluster'] = KMeans(n_clusters = clusters).fit(np.array(merged['angle']).reshape(-1, 1)).labels_

array([1, 2, 1, 0, 0, 2, 0, 1, 1, 0, 0, 2, 1, 1, 0], dtype=int32)

In [ ]:
merged.columns = ['restaurant', 'lat_rest', 'long_rest', 'customer', 'lat_cust', 'long_cust',
       'measure', 'cluster']

In [ ]:
for repeat in repeatsRest.index:
    print(repeat)
    test = merged.index[merged['restaurant'] == repeat].tolist()

    p = 0
    for index in test:
        merged.loc[merged.index == index,  'restaurant'] = repeat + '_' + str(p)
        p += 1

for repeat in repeatsCust.index:
    print(repeat)
    test = merged.index[merged['customer'] == repeat].tolist()

    p = 0
    for index in test:
        merged.loc[merged.index == index,  'customer'] = repeat + '_' + str(p)
        p += 1

Downtown Toronto (St. James Park)
Downtown Toronto (Rosedale)
Downtown Toronto (Central Bay Street)


In [ ]:
#Function to get arrows for route lines
def getArrows(locations, color='blue', size=6, n_arrows=3):
    
    '''
    Get a list of placed and rotated arrows or markers to be plotted
    
    Parameters
    locations : list of lists of latitude longitude that represent the begining and end of Line. 
                    this function Return list of arrows or the markers
    '''
    
    Point = namedtuple('Point', field_names=['lat', 'lon'])
    
    # creating point from Point named tuple
    point1 = Point(locations[0][0], locations[0][1])
    point2 = Point(locations[1][0], locations[1][1])
    
    # calculate the rotation required for the marker.  
    #Reducing 90 to account for the orientation of marker
    # Get the degree of rotation
    angle = get_angle(point1, point2) - 90
    
    # get the evenly space list of latitudes and longitudes for the required arrows

    arrow_latitude = np.linspace(point1.lat, point2.lat, n_arrows + 2)[1:n_arrows+1]
    arrow_longitude = np.linspace(point1.lon, point2.lon, n_arrows + 2)[1:n_arrows+1]
    
    final_arrows = []
    
    #creating each "arrow" and appending them to our arrows list
    for points in zip(arrow_latitude, arrow_longitude):
        final_arrows.append(folium.RegularPolygonMarker(location=points, 
                      fill_color=color, number_of_sides=3, 
                      radius=size, rotation=angle))
    return final_arrows

In [ ]:
#Function to get arrows angle for route lines
def get_angle(p1, p2):
    
    '''
    This function Returns angle value in degree from the location p1 to location p2
    
    Parameters it accepts : 
    p1 : namedtuple with lat lon
    p2 : namedtuple with lat lon
    
    This function Return the vlaue of degree in the data type float
    
    Pleae also refers to for better understanding : https://gist.github.com/jeromer/2005586
    '''
    
    longitude_diff = np.radians(p2.lon - p1.lon)
    
    latitude1 = np.radians(p1.lat)
    latitude2 = np.radians(p2.lat)
    
    x_vector = np.sin(longitude_diff) * np.cos(latitude2)
    y_vector = (np.cos(latitude1) * np.sin(latitude2) 
        - (np.sin(latitude1) * np.cos(latitude2) 
        * np.cos(longitude_diff)))
    angle = np.degrees(np.arctan2(x_vector, y_vector))
    
    # Checking and adjustring angle value on the scale of 360
    if angle < 0:
        return angle + 360
    return angle

## Optimizing to determine Driver Assignment

In [ ]:
maps = []
SumDis = 0

colors = ['red','blue','orange','pink']

ClusterWait = 0
ClusterCust = 0
m= folium.Map(location=[43.6532,  -79.3832])
OptimalStart = []


for cluster in range(0,clusters):

    select = merged[merged['cluster'] == cluster]

    df = df_4.merge(select, on = ['restaurant','customer'], how = 'inner')

    W = 1000

    start = list(df["restaurant"].unique()) + list(df["customer"].unique())
    #start.append("Downtown Toronto (Rosedale)")
    start = list(dict.fromkeys(start))

    end = list(df["restaurant"].unique()) + list(df["customer"].unique())
    end = list(dict.fromkeys(end))
    
    step = np.arange(0, len(end)).tolist()

    restaurants = df["restaurant"].unique().tolist()

    customers = df["customer"].unique().tolist()

    list_order = []
    for index, row in df.iterrows():
        list_order.append([row["restaurant"], row["customer"]])
    print(list_order)

    location_distance = {}
    for i in start:
        for j in end: 
            if i.split('_')[0] == j.split('_')[0]:
                location_distance[(i,j)] = 0 
            else:
                try:
                    location_distance[(i,j)] = float(df_distance[(df_distance["origin"] == i)&(df_distance["destination"] == j)]["distance"])
                except:
                    temp = df_distance[(df_distance["origin"] == i.split('_')[0])&(df_distance["destination"] == j.split('_')[0])]["distance"]

                    #print(i.split('_')[0])
                    #print(j.split('_')[0])
                    location_distance[(i,j)] = float(df_distance[(df_distance["origin"] == i.split('_')[0])&(df_distance["destination"] == j.split('_')[0])]["distance"])
                
    min_arrival_time_mins = {}
    for j in end:
        if j in restaurants:
            delta = pd.to_datetime(df.loc[df["restaurant"] == j]["estimated availability"].tolist()[0]) - start_time
            min_arrival_time_mins[j] = delta/datetime.timedelta(minutes=1)
        if j in customers: 
            #delta = pd.to_datetime(df_2.loc[df_2["customer"] == j]["estimated availability"].tolist()[0]) - start_time
            #min_arrival_time_mins[j] = delta/datetime.timedelta(minutes=1)
            min_arrival_time_mins[j] = 0 

    
    wait_time = 5
    velocity = 1/3 # 20 km/h = 1/3 km/min
    travel_time = {}
    for i in start:
        for j in end:
            if i == j:
                travel_time[(i,j)] = 0 
            else: 
                if i in customers:
                    travel_time[(i,j)] = location_distance[(i,j)]*velocity + wait_time
                else:
                    travel_time[(i,j)] = location_distance[(i,j)]*velocity 
    
    xvar = pulp.LpVariable.dict("x", (start, end, step), cat=pulp.LpBinary)
    dvar = pulp.LpVariable.dict("d", step, lowBound=0.0, cat=pulp.LpContinuous)
    wvar = pulp.LpVariable.dict("w", (end, step), lowBound=0.0, cat=pulp.LpContinuous)
    zvar = pulp.LpVariable.dict("z", step, lowBound=0.0, cat=pulp.LpContinuous)


    flowmodel = pulp.LpProblem(name = "FlowModel", sense = pulp.LpMinimize)

    # Constraint: every location visited once
    for j in end: 
        flowmodel += pulp.lpSum([xvar[(i,j,t)] for i in start for t in step]) == 1
        
        
    # Constraint: Convervation of flow
    for t in step[:-1]:
        for j in end:
            flowmodel += (pulp.lpSum([xvar[(i,j,t)] for i in start]) == pulp.lpSum([xvar[(j,k,t+1)] for k in end]))

            
    # Constraint: First node (Rosedale) has an outflow of 1 in the first step, others are zero
    #for i in start:
        #if i == "Downtown Toronto (Rosedale)":
            #flowmodel += pulp.lpSum([xvar[(i,j,0)] for j in end]) == 1
        #else: 
            #flowmodel += pulp.lpSum([xvar[(i,j,0)] for j in end]) == 0

            
    def find_restaurant_of_customer(customer):
        for order in list_order:
            if customer == order[1]:
                return order[0]
            
    # Constraint: make sure driver goes to a restaurant before delivering the food
    for t in step: 
        for j in end: 
            if j in customers: 
                flowmodel += pulp.lpSum( [xvar[i,j,t] for i in start]) <= pulp.lpSum([xvar[i, find_restaurant_of_customer(j),t_] for i in start for t_ in step[:t]])
            

    # Constraint: Time to arrive at the location of the t-th step
    for t in step:
        if t == 0: 
            flowmodel += dvar[t] == pulp.lpSum([ xvar[(i,j,0)] * min_arrival_time_mins[j] for i in start for j in end])
        else:
            flowmodel += dvar[t] >= (dvar[t-1] + pulp.lpSum([ xvar[(i,j,t)] * travel_time[(i,j)] for i in start for j in end]))


            
    # Constraint: if arriving at location j, ensures time is at least the min_arrival_time_mins
    for t in step:
        for i in start:
            for j in end:
                flowmodel += dvar[t] >= min_arrival_time_mins[j] * xvar[i,j,t]
            
            
    # Constraint: define waiting time of each order at step t: wvar[j,t]
    for t in step:
        for j in end:
            if j in restaurants:
                flowmodel += wvar[(j,t)]  == 0
            else:
                flowmodel += wvar[(j,t)] == dvar[t] - pulp.lpSum([ min_arrival_time_mins[find_restaurant_of_customer(j)] * xvar[(i,j,t)] for i in start])

                
    # # Constraint: ensures the average waiting time of the orders is lower than the maximum W

    # implementation idea: 
    #         flowmodel += pulp.lpSum([wvar[j,t] * xvar[i,j,t] for i in start]) <= W
    # use zvar[t] with big Constant M to denote wvar[j,t] * xvar[i,j,t] as two decision variables cannot be multiplied together
    # zvar[t] is the total waiting time up until the step t

    M = 500000
    for t in step:
        for i in start:
            for j in end:
                if t == 0:
                    flowmodel += zvar[t] == 0 
                elif t == 1:
                    flowmodel += zvar[t] >= wvar[j,t] - M * (1 - xvar[(i,j,t)])
                    flowmodel += zvar[t] <= wvar[j,t] + M * (1 - xvar[(i,j,t)])
                else: 
                    flowmodel += zvar[t] >= zvar[t-1] + wvar[(j,t)] - M * (1 - xvar[(i,j,t)])
                    flowmodel += zvar[t] <= zvar[t-1] + wvar[(j,t)] + M * (1 - xvar[(i,j,t)])

                    
    for t in step:
        flowmodel += zvar[t] <= W * len(customers)

    obj = pulp.lpSum([location_distance[(i,j)] * xvar[(i,j,t)] for i in start for j in end for t in step])
    flowmodel += obj

    flowmodel.solve()
    print("Status:", pulp.LpStatus[flowmodel.status])

    # Total Distance
    totalDistance = pulp.value(flowmodel.objective)
    print("Total Distance: " + str(totalDistance))

    # print solution

    r = 0
    for t in step: 
        print("Step: " + str(t))
        print("Total travel time (in minutes) up to this step: " + str(dvar[t].varValue))
        
        for i in start: 
            for j in end: 
                if xvar[(i,j,t)].varValue > 0.0:

                    if r == 0:
                        OptimalStart.append(i)
                    print("\tLeave from " + str(i))
                    print("\tTravel to " + str(j))
                    print("\tWaiting time: " + str(wvar[(j,t)].varValue))
                    print("\tTotal waiting time up to this step: " + str(zvar[t].varValue))
                    r +=1
                    
    print("Average waiting time: " + str(zvar[t].varValue/len(customers)))

    ClusterWait += zvar[t].varValue
    ClusterCust += len(customers)

    loc_i=[]
    loc_j=[]
    for t in step: 
        for i in start:
            for j in end:
                if xvar[(i,j,t)].varValue > 0.0:
                    loc_j.append(j)
                    loc_i.append(i)

    loc_i=pd.DataFrame(loc_i, columns=['loc_i'])
    loc_j=pd.DataFrame(loc_j, columns=['loc_j'])
    steps=pd.DataFrame(step, columns=['steps'])

    dfviz = pd.concat([loc_i, loc_j],axis=1, join='inner')
    dfviz = pd.concat([dfviz, steps], axis=1, join='inner')
    
    #new1 = dfviz['loc_i'].str.split('_')
    #dfviz['Tempi'] = [item[0] for item in new1]

    new2 = dfviz['loc_j'].str.split('_')
    dfviz['Tempj'] = [item[0] for item in new2]

    new2 = dfviz['loc_i'].str.split('_')
    dfviz['Tempi'] = [item[0] for item in new2]

    dfviz2 = dfviz.merge(regions[['name', 'latitude', 'longitude']], how = 'left',
                left_on = 'Tempi', right_on = 'name').drop(columns = ['name'])

    dfviz2=dfviz2.rename(columns={"latitude": "lat_i", "longitude": "long_i"})

    dfviz2 = dfviz2.merge(regions[['name', 'latitude', 'longitude']], how = 'left',
                left_on = 'Tempj', right_on = 'name').drop(columns = ['name'])

    dfviz2=dfviz2.rename(columns={"latitude": "lat_j", "longitude": "long_j"})

    #m= folium.Map(location=[43.6532,  -79.3832])
    marker_icon='cloud'
    popup='Default popup Marker1'

    #Setting location markers for locations i
    for i in range(0, len(dfviz2.index)):
        folium.Marker(location=[dfviz2['lat_i'][i],  dfviz2['long_i'][i]],tooltip=dfviz2['loc_i'][i],icon=folium.Icon(color="green")).add_to(m)

    #Setting location markers for locations j
    for i in range(0, len(dfviz2.index)):
        folium.Marker(location=[dfviz2['lat_j'][i],  dfviz2['long_j'][i]],tooltip=dfviz2['loc_j'][i],icon=folium.Icon(color="red")).add_to(m)

    route = []

    route.append([dfviz2['lat_i'][0], dfviz2['long_i'][0]])

    for i in range(0, len(dfviz2.index)):
        route.append([dfviz2['lat_j'][i], dfviz2['long_j'][i]])

    #Defining a feature group

    string = "Route" + str(cluster)

    f1=folium.FeatureGroup("string")



    line_1=folium.vector_layers.PolyLine(route,popup='<b>Path of Vehicle_1</b>',sticky=True,tooltip='Route 1',color=colors[cluster],weight=3).add_to(f1)

    for i in range(0, len(route)-1):
        loc1=route[i]
        loc2=route[i+1]
        arrows = getArrows(locations=[loc1,loc2], n_arrows=3)
        for arrow in arrows:
            arrow.add_to(m)

    f1.add_to(m)
    maps.append(m)

    SumDis += totalDistance

[['Downtown Toronto (St. James Park)_0', 'East Toronto (The Beaches)'], ['Downtown Toronto (Kensington Market / Chinatown / Grange Park)', 'Downtown Toronto (Central Bay Street)_0'], ['Downtown Toronto (Regent Park / Port of Toronto)', 'Downtown Toronto (Toronto Dominion Centre / Design Exchange)'], ['Downtown Toronto (Church and Wellesley)', 'East Toronto (The Danforth West / Riverdale)'], ['Downtown Toronto (Richmond / Adelaide / King)', 'East York (Parkview Hill / Woodbine Gardens)'], ['Downtown Toronto (St. James Park)_2', 'East Toronto (India Bazaar / The Beaches West)']]
Status: Optimal
Total Distance: 16.997212185635025
Step: 0
Total travel time (in minutes) up to this step: 81.0
	Leave from Downtown Toronto (Regent Park / Port of Toronto)
	Travel to Downtown Toronto (Regent Park / Port of Toronto)
	Waiting time: 0.0
	Total waiting time up to this step: 0.0
Step: 1
Total travel time (in minutes) up to this step: 144.0
	Leave from Downtown Toronto (Regent Park / Port of Toronto)


In [ ]:
print("Distance:",SumDis)
print("Average Time:", ClusterWait/ClusterCust)

Distance: 61.28131805770934
Average Time: 74.95517133333333


In [ ]:
drivers = pd.read_csv('part4_drivers.csv')
#drivers = pd.read_csv('part3_drivers.csv')
drivers

,start region,velocity
0,Downtown Toronto (Richmond / Adelaide / King),20
1,Downtown Toronto (St. James Park),20
2,Downtown Toronto (Church and Wellesley),20
3,Downtown Toronto (Christie),20
4,Downtown Toronto (Church and Wellesley),20
5,Downtown Toronto Stn A PO Boxes 25 The Esplana...,20
6,Downtown Toronto (CN Tower / King and Spadina ...,20
7,Downtown Toronto (Rosedale),20
8,Downtown Toronto (University of Toronto / Harb...,20
9,Downtown Toronto (Ryerson),20


In [ ]:
OptimalStart
drivers2 = drivers

drivers2 =drivers2.merge(regions, left_on = 'start region', right_on = 'name')
drivers2=drivers2.rename(columns={"latitude": "driver_lat", "longitude": "driver_long"})


for start in OptimalStart:
    start = start.split('_')[0]

    lat = float(regions[regions['name'] == start]['latitude'])
    long = float(regions[regions['name'] == start]['longitude'])

    name = start + '_Dist'
    drivers2[name] = ((lat - drivers2['driver_lat'])**2) +((long - drivers2['driver_long'])**2)
drivers2


driverList = []
for start in OptimalStart:
    start = start.split('_')[0]


    driver = drivers2[drivers2[start +'_Dist'] == drivers2[start +'_Dist'].min()]['start region'].iloc[0]

    if driver not in driverList:
        driverList.append(driver)
    else:
        for i in range(2,1000):
            driver = drivers2[drivers2[start +'_Dist'] == drivers2[start +'_Dist'].nsmallest(i).iloc[-1]]['start region'].iloc[0]
            if driver not in driverList:
                driverList.append(driver)
                break
driverList

['Downtown Toronto (St. James Park)',
 'Downtown Toronto (Church and Wellesley)',
 'Downtown Toronto (Rosedale)']

## Optimizing Route With Drivers

In [ ]:
maps = []
SumDis = 0

colors = ['red','blue','orange','pink']

ClusterWait = 0
ClusterCust = 0
m= folium.Map(location=[43.6532,  -79.3832])
OptimalStart = []


for cluster in range(0,clusters):
    
    select = merged[merged['cluster'] == cluster]

    df = df_4.merge(select, on = ['restaurant','customer'], how = 'inner')

    W = 120

    start = list(df["restaurant"].unique()) + list(df["customer"].unique())
    start.append(driverList[cluster])
    start = list(dict.fromkeys(start))

    end = list(df["restaurant"].unique()) + list(df["customer"].unique())
    end = list(dict.fromkeys(end))
    
    step = np.arange(0, len(end)).tolist()

    restaurants = df["restaurant"].unique().tolist()

    customers = df["customer"].unique().tolist()

    list_order = []
    for index, row in df.iterrows():
        list_order.append([row["restaurant"], row["customer"]])
    print(list_order)

    location_distance = {}
    for i in start:
        for j in end: 
            if i.split('_')[0] == j.split('_')[0]:
                location_distance[(i,j)] = 0 
            else:
                try:
                    location_distance[(i,j)] = float(df_distance[(df_distance["origin"] == i)&(df_distance["destination"] == j)]["distance"])
                except:
                    temp = df_distance[(df_distance["origin"] == i.split('_')[0])&(df_distance["destination"] == j.split('_')[0])]["distance"]

                    #print(i.split('_')[0])
                    #print(j.split('_')[0])
                    location_distance[(i,j)] = float(df_distance[(df_distance["origin"] == i.split('_')[0])&(df_distance["destination"] == j.split('_')[0])]["distance"])
                
    min_arrival_time_mins = {}
    for j in end:
        if j in restaurants:
            delta = pd.to_datetime(df.loc[df["restaurant"] == j]["estimated availability"].tolist()[0]) - start_time
            min_arrival_time_mins[j] = delta/datetime.timedelta(minutes=1)
        if j in customers: 
            #delta = pd.to_datetime(df_2.loc[df_2["customer"] == j]["estimated availability"].tolist()[0]) - start_time
            #min_arrival_time_mins[j] = delta/datetime.timedelta(minutes=1)
            min_arrival_time_mins[j] = 0 

    
    wait_time = 5
    velocity = int(drivers[drivers['start region'] == driverList[cluster]]['velocity'].iloc[0])/60
    #velocity = 1/3 # 20 km/h = 1/3 km/min
    travel_time = {}
    for i in start:
        for j in end:
            if i == j:
                travel_time[(i,j)] = 0 
            else: 
                if i in customers:
                    travel_time[(i,j)] = location_distance[(i,j)]*velocity + wait_time
                else:
                    travel_time[(i,j)] = location_distance[(i,j)]*velocity 
    
    xvar = pulp.LpVariable.dict("x", (start, end, step), cat=pulp.LpBinary)
    dvar = pulp.LpVariable.dict("d", step, lowBound=0.0, cat=pulp.LpContinuous)
    wvar = pulp.LpVariable.dict("w", (end, step), lowBound=0.0, cat=pulp.LpContinuous)
    zvar = pulp.LpVariable.dict("z", step, lowBound=0.0, cat=pulp.LpContinuous)


    flowmodel = pulp.LpProblem(name = "FlowModel", sense = pulp.LpMinimize)

    # Constraint: every location visited once
    for j in end: 
        flowmodel += pulp.lpSum([xvar[(i,j,t)] for i in start for t in step]) == 1
        
        
    # Constraint: Convervation of flow
    for t in step[:-1]:
        for j in end:
            flowmodel += (pulp.lpSum([xvar[(i,j,t)] for i in start]) == pulp.lpSum([xvar[(j,k,t+1)] for k in end]))

            
     #Constraint: First node (Rosedale) has an outflow of 1 in the first step, others are zero
    for i in start:
        if i == driverList[cluster]:
            flowmodel += pulp.lpSum([xvar[(i,j,0)] for j in end]) == 1
        else: 
            flowmodel += pulp.lpSum([xvar[(i,j,0)] for j in end]) == 0

            
    def find_restaurant_of_customer(customer):
        for order in list_order:
            if customer == order[1]:
                return order[0]
            
    # Constraint: make sure driver goes to a restaurant before delivering the food
    for t in step: 
        for j in end: 
            if j in customers: 
                flowmodel += pulp.lpSum( [xvar[i,j,t] for i in start]) <= pulp.lpSum([xvar[i, find_restaurant_of_customer(j),t_] for i in start for t_ in step[:t]])
            

    # Constraint: Time to arrive at the location of the t-th step
    for t in step:
        if t == 0: 
            flowmodel += dvar[t] == pulp.lpSum([ xvar[(i,j,0)] * min_arrival_time_mins[j] for i in start for j in end])
        else:
            flowmodel += dvar[t] >= (dvar[t-1] + pulp.lpSum([ xvar[(i,j,t)] * travel_time[(i,j)] for i in start for j in end]))


            
    # Constraint: if arriving at location j, ensures time is at least the min_arrival_time_mins
    for t in step:
        for i in start:
            for j in end:
                flowmodel += dvar[t] >= min_arrival_time_mins[j] * xvar[i,j,t]
            
            
    # Constraint: define waiting time of each order at step t: wvar[j,t]
    for t in step:
        for j in end:
            if j in restaurants:
                flowmodel += wvar[(j,t)]  == 0
            else:
                flowmodel += wvar[(j,t)] == dvar[t] - pulp.lpSum([ min_arrival_time_mins[find_restaurant_of_customer(j)] * xvar[(i,j,t)] for i in start])

                
    # # Constraint: ensures the average waiting time of the orders is lower than the maximum W

    # implementation idea: 
    #         flowmodel += pulp.lpSum([wvar[j,t] * xvar[i,j,t] for i in start]) <= W
    # use zvar[t] with big Constant M to denote wvar[j,t] * xvar[i,j,t] as two decision variables cannot be multiplied together
    # zvar[t] is the total waiting time up until the step t

    M = 500000
    for t in step:
        for i in start:
            for j in end:
                if t == 0:
                    flowmodel += zvar[t] == 0 
                elif t == 1:
                    flowmodel += zvar[t] >= wvar[j,t] - M * (1 - xvar[(i,j,t)])
                    flowmodel += zvar[t] <= wvar[j,t] + M * (1 - xvar[(i,j,t)])
                else: 
                    flowmodel += zvar[t] >= zvar[t-1] + wvar[(j,t)] - M * (1 - xvar[(i,j,t)])
                    flowmodel += zvar[t] <= zvar[t-1] + wvar[(j,t)] + M * (1 - xvar[(i,j,t)])

                    
    for t in step:
        flowmodel += zvar[t] <= W * len(customers)

    obj = pulp.lpSum([location_distance[(i,j)] * xvar[(i,j,t)] for i in start for j in end for t in step])
    flowmodel += obj

    flowmodel.solve()
    print("Status:", pulp.LpStatus[flowmodel.status])

    # Total Distance
    totalDistance = pulp.value(flowmodel.objective)
    print("Total Distance: " + str(totalDistance))

    # print solution

    r = 0
    for t in step: 
        print("Step: " + str(t))
        print("Total travel time (in minutes) up to this step: " + str(dvar[t].varValue))
        
        for i in start: 
            for j in end: 
                if xvar[(i,j,t)].varValue > 0.0:

                    if r == 0:
                        OptimalStart.append(i)
                    print("\tLeave from " + str(i))
                    print("\tTravel to " + str(j))
                    print("\tWaiting time: " + str(wvar[(j,t)].varValue))
                    print("\tTotal waiting time up to this step: " + str(zvar[t].varValue))
                    r +=1
                    
    print("Average waiting time: " + str(zvar[t].varValue/len(customers)))

    ClusterWait += zvar[t].varValue
    ClusterCust += len(customers)

    loc_i=[]
    loc_j=[]
    for t in step: 
        for i in start:
            for j in end:
                if xvar[(i,j,t)].varValue > 0.0:
                    loc_j.append(j)
                    loc_i.append(i)

    loc_i=pd.DataFrame(loc_i, columns=['loc_i'])
    loc_j=pd.DataFrame(loc_j, columns=['loc_j'])
    steps=pd.DataFrame(step, columns=['steps'])

    dfviz = pd.concat([loc_i, loc_j],axis=1, join='inner')
    dfviz = pd.concat([dfviz, steps], axis=1, join='inner')
    
    #new1 = dfviz['loc_i'].str.split('_')
    #dfviz['Tempi'] = [item[0] for item in new1]

    new2 = dfviz['loc_j'].str.split('_')
    dfviz['Tempj'] = [item[0] for item in new2]

    new2 = dfviz['loc_i'].str.split('_')
    dfviz['Tempi'] = [item[0] for item in new2]

    dfviz2 = dfviz.merge(regions[['name', 'latitude', 'longitude']], how = 'left',
                left_on = 'Tempi', right_on = 'name').drop(columns = ['name'])

    dfviz2=dfviz2.rename(columns={"latitude": "lat_i", "longitude": "long_i"})

    dfviz2 = dfviz2.merge(regions[['name', 'latitude', 'longitude']], how = 'left',
                left_on = 'Tempj', right_on = 'name').drop(columns = ['name'])

    dfviz2=dfviz2.rename(columns={"latitude": "lat_j", "longitude": "long_j"})

    #m= folium.Map(location=[43.6532,  -79.3832])
    marker_icon='cloud'
    popup='Default popup Marker1'

    #Setting location markers for locations i
    for i in range(0, len(dfviz2.index)):
        folium.Marker(location=[dfviz2['lat_i'][i],  dfviz2['long_i'][i]],tooltip=dfviz2['loc_i'][i],icon=folium.Icon(color="green")).add_to(m)

    #Setting location markers for locations j
    for i in range(0, len(dfviz2.index)):
        folium.Marker(location=[dfviz2['lat_j'][i],  dfviz2['long_j'][i]],tooltip=dfviz2['loc_j'][i],icon=folium.Icon(color="red")).add_to(m)

    route = []

    route.append([dfviz2['lat_i'][0], dfviz2['long_i'][0]])

    for i in range(0, len(dfviz2.index)):
        route.append([dfviz2['lat_j'][i], dfviz2['long_j'][i]])

    #Defining a feature group

    string = "Route" + str(cluster)

    f1=folium.FeatureGroup("string")



    line_1=folium.vector_layers.PolyLine(route,popup='<b>Path of Vehicle_1</b>',sticky=True,tooltip='Route 1',color=colors[cluster],weight=3).add_to(f1)

    for i in range(0, len(route)-1):
        loc1=route[i]
        loc2=route[i+1]
        arrows = getArrows(locations=[loc1,loc2], n_arrows=3)
        for arrow in arrows:
            arrow.add_to(m)

    f1.add_to(m)
    maps.append(m)

    SumDis += totalDistance

[['Downtown Toronto (St. James Park)_0', 'East Toronto (The Beaches)'], ['Downtown Toronto (Kensington Market / Chinatown / Grange Park)', 'Downtown Toronto (Central Bay Street)_0'], ['Downtown Toronto (Regent Park / Port of Toronto)', 'Downtown Toronto (Toronto Dominion Centre / Design Exchange)'], ['Downtown Toronto (Church and Wellesley)', 'East Toronto (The Danforth West / Riverdale)'], ['Downtown Toronto (Richmond / Adelaide / King)', 'East York (Parkview Hill / Woodbine Gardens)'], ['Downtown Toronto (St. James Park)_2', 'East Toronto (India Bazaar / The Beaches West)']]
Status: Optimal
Total Distance: 18.120334430240636
Step: 0
Total travel time (in minutes) up to this step: 144.0
	Leave from Downtown Toronto (St. James Park)
	Travel to Downtown Toronto (St. James Park)_2
	Waiting time: 0.0
	Total waiting time up to this step: 0.0
Step: 1
Total travel time (in minutes) up to this step: 144.0
	Leave from Downtown Toronto (St. James Park)_2
	Travel to Downtown Toronto (St. James P

In [ ]:
print("Distance:",SumDis)
print("Average Time:", ClusterWait/ClusterCust)

Distance: 63.580528399141556
Average Time: 75.25764333333333


In [ ]:
m

# Multiple Drivers per Cluster

In [ ]:
df_distance = pd.read_csv("distances.csv")
df_drivers4 = pd.read_csv("part4_drivers.csv")
df_4 = pd.read_csv("part4_large.csv")
regions = pd.read_csv("regions.csv")

start_time = pd.to_datetime('2022-04-02 17:00:00')
W = 120

In [ ]:
df_drivers4

,start region,velocity
0,Downtown Toronto (Richmond / Adelaide / King),20
1,Downtown Toronto (St. James Park),20
2,Downtown Toronto (Church and Wellesley),20
3,Downtown Toronto (Christie),20
4,Downtown Toronto (Church and Wellesley),20
5,Downtown Toronto Stn A PO Boxes 25 The Esplana...,20
6,Downtown Toronto (CN Tower / King and Spadina ...,20
7,Downtown Toronto (Rosedale),20
8,Downtown Toronto (University of Toronto / Harb...,20
9,Downtown Toronto (Ryerson),20


In [ ]:
occur1 = df_4['restaurant'].value_counts()
repeatsRest = occur1[occur1 > 1]

occur2 = df_4['customer'].value_counts()
repeatsCust = occur2[occur2 > 1]

In [ ]:
df_4_rest = pd.DataFrame(df_4['restaurant'])

df_4_rest = df_4_rest.rename(columns={"restaurant": "name"})

df_4_cust = pd.DataFrame(df_4[['customer']])
df_4_cust = df_4_cust.rename(columns={"customer": "name"})

In [ ]:
for repeat in repeatsRest.index:
    print(repeat)
    test = df_4.index[df_4['restaurant'] == repeat].tolist()

    p = 0
    for index in test:
        df_4.loc[df_4.index == index,  'restaurant'] = repeat + '_' + str(p)
        p += 1

for repeat in repeatsCust.index:
    print(repeat)
    test = df_4.index[df_4['customer'] == repeat].tolist()

    p = 0
    for index in test:
        df_4.loc[df_4.index == index,  'customer'] = repeat + '_' + str(p)
        p += 1

Downtown Toronto (St. James Park)
Downtown Toronto (Rosedale)
Downtown Toronto (Central Bay Street)


In [ ]:
regions = regions[['name','latitude','longitude']]

In [ ]:
merged_cust = df_4_cust.merge(regions, on = 'name', how = 'left')
merged_rest = df_4_rest.merge(regions, on = 'name', how = 'left')

merged_cust = merged_cust.rename(columns={"latitude": "lat_cust", "longitude": "long_cust"})
merged_rest = merged_rest.rename(columns={"latitude": "lat_rest", "longitude": "long_rest"})

In [ ]:
merged = pd.concat([merged_rest, merged_cust], axis=1)

In [ ]:
merged

,name,lat_rest,long_rest,name,lat_cust,long_cust
0,Downtown Toronto (Central Bay Street),43.6564,-79.3860,North York (Armour Heights / Wilson Heights / ...,43.7535,-79.4472
1,Downtown Toronto (Christie),43.6683,-79.4205,West Toronto (Brockton / Parkdale Village / Ex...,43.6383,-79.4301
2,Downtown Toronto (Ryerson),43.6572,-79.3783,York (Fairbank / Oakwood),43.6889,-79.4507
3,Downtown Toronto (St. James Park),43.6513,-79.3756,East Toronto (The Beaches),43.6784,-79.2941
4,Downtown Toronto (Kensington Market / Chinatow...,43.6541,-79.3978,Downtown Toronto (Central Bay Street),43.6564,-79.3860
5,Downtown Toronto (Rosedale),43.6827,-79.3730,Downtown Toronto (CN Tower / King and Spadina ...,43.6404,-79.3995
6,Downtown Toronto (Regent Park / Port of Toronto),43.6555,-79.3626,Downtown Toronto (Toronto Dominion Centre / De...,43.6469,-79.3823
7,Downtown Toronto (St. James Town / Cabbagetown),43.6684,-79.3689,York (Runnymede / The Junction North),43.6748,-79.4839
8,Downtown Toronto (Underground city),43.6492,-79.3823,Central Toronto (Summerhill West / Rathnelly /...,43.6861,-79.4025
9,Downtown Toronto (Church and Wellesley),43.6656,-79.3830,East Toronto (The Danforth West / Riverdale),43.6803,-79.3538


In [ ]:
centroid_Lat = merged['lat_rest'].mean()
centroid_Long = merged['long_rest'].mean()

### Define Clustering

In [ ]:
# Cluster by angle between restaurant and Customer
merged['measure'] = np.arctan((merged['long_cust'] - merged['long_rest'])/(merged['lat_cust'] - merged['lat_rest']))

# Cluster by angle between resturant centroid and customer
# merged['measure'] = np.arctan((merged['long_cust'] - centroid_Long)/(merged['lat_cust'] - centroid_Lat))

#Cluster by Radius 
# merged['measure'] = np.sqrt((merged['long_cust'] - centroid_Long)**2/(merged['lat_cust'] - centroid_Lat)**2)

In [ ]:
clusters = 3

clf = KMeansConstrained(
     n_clusters=clusters,
     size_min=2,
     size_max=6)

In [ ]:
clusterList = clf.fit_predict(np.array(merged['measure']).reshape(-1, 1))
merged['cluster'] = clf.labels_
clusterList
#merged['cluster'] = KMeans(n_clusters = clusters).fit(np.array(merged['angle']).reshape(-1, 1)).labels_

array([0, 2, 0, 1, 1, 2, 1, 0, 0, 1, 1, 2, 0, 0, 1], dtype=int32)

In [ ]:
merged.columns = ['restaurant', 'lat_rest', 'long_rest', 'customer', 'lat_cust', 'long_cust',
       'measure', 'cluster']

In [ ]:
for repeat in repeatsRest.index:
    print(repeat)
    test = merged.index[merged['restaurant'] == repeat].tolist()

    p = 0
    for index in test:
        merged.loc[merged.index == index,  'restaurant'] = repeat + '_' + str(p)
        p += 1

for repeat in repeatsCust.index:
    print(repeat)
    test = merged.index[merged['customer'] == repeat].tolist()

    p = 0
    for index in test:
        merged.loc[merged.index == index,  'customer'] = repeat + '_' + str(p)
        p += 1

Downtown Toronto (St. James Park)
Downtown Toronto (Rosedale)
Downtown Toronto (Central Bay Street)


In [ ]:
#Function to get arrows for route lines
def getArrows(locations, color='blue', size=6, n_arrows=3):
    
    '''
    Get a list of placed and rotated arrows or markers to be plotted
    
    Parameters
    locations : list of lists of latitude longitude that represent the begining and end of Line. 
                    this function Return list of arrows or the markers
    '''
    
    Point = namedtuple('Point', field_names=['lat', 'lon'])
    
    # creating point from Point named tuple
    point1 = Point(locations[0][0], locations[0][1])
    point2 = Point(locations[1][0], locations[1][1])
    
    # calculate the rotation required for the marker.  
    #Reducing 90 to account for the orientation of marker
    # Get the degree of rotation
    angle = get_angle(point1, point2) - 90
    
    # get the evenly space list of latitudes and longitudes for the required arrows

    arrow_latitude = np.linspace(point1.lat, point2.lat, n_arrows + 2)[1:n_arrows+1]
    arrow_longitude = np.linspace(point1.lon, point2.lon, n_arrows + 2)[1:n_arrows+1]
    
    final_arrows = []
    
    #creating each "arrow" and appending them to our arrows list
    for points in zip(arrow_latitude, arrow_longitude):
        final_arrows.append(folium.RegularPolygonMarker(location=points, 
                      fill_color=color, number_of_sides=3, 
                      radius=size, rotation=angle))
    return final_arrows

In [ ]:
#Function to get arrows angle for route lines
def get_angle(p1, p2):
    
    '''
    This function Returns angle value in degree from the location p1 to location p2
    
    Parameters it accepts : 
    p1 : namedtuple with lat lon
    p2 : namedtuple with lat lon
    
    This function Return the vlaue of degree in the data type float
    
    Pleae also refers to for better understanding : https://gist.github.com/jeromer/2005586
    '''
    
    longitude_diff = np.radians(p2.lon - p1.lon)
    
    latitude1 = np.radians(p1.lat)
    latitude2 = np.radians(p2.lat)
    
    x_vector = np.sin(longitude_diff) * np.cos(latitude2)
    y_vector = (np.cos(latitude1) * np.sin(latitude2) 
        - (np.sin(latitude1) * np.cos(latitude2) 
        * np.cos(longitude_diff)))
    angle = np.degrees(np.arctan2(x_vector, y_vector))
    
    # Checking and adjustring angle value on the scale of 360
    if angle < 0:
        return angle + 360
    return angle

# Multiple Drivers 

In [ ]:
maps = []
SumDis = 0

colors = ['red','blue','orange','pink']

ClusterWait = 0
ClusterCust = 0
m= folium.Map(location=[43.6532,  -79.3832])
OptimalStart = []

In [ ]:
# available = [0:9]
# vehicle = []
# for i in range(clusters):
#     sublist = []
#     chosen = random.sample(range(0, 9), 2)
#     sublist.append()
#     listoflists.append(sublist)

In [ ]:
for cluster in range(0,clusters):
    
    select = merged[merged['cluster'] == cluster]
    df = df_4.merge(select, on = ['restaurant','customer'], how = 'inner')
    
    vehicle = np.arange(0, len(df_drivers4)).tolist()
    vehicle = vehicle[cluster*2:cluster*2+2]
    
    print(vehicle)
    
    start_region = []
    for v in vehicle:
        start_region.append(df_drivers4["start region"][v])
    
    W = 1000

    restaurant = df["restaurant"].unique().tolist()

    customer = df["customer"].unique().tolist()

    loc_index = {}
    locations = start_region + restaurant + customer

    for i, location in enumerate(locations):
        loc_index[i] = location

    start = np.arange(0, len(loc_index)).tolist()

    end = start[len(start_region):]

    start_regions = start[:len(start_region)]

    restaurants = start[len(start_region):-len(customer)]

    customers = start[-len(customer):]

    step = np.arange(0, len(locations)).tolist()

    list_order = []
    for index, row in df.iterrows():
        list_order.append([row["restaurant"], row["customer"]])

    #     a = df_distance[(df_distance["origin"] == loc_index[0]) & (df_distance["destination"] == loc_index[j])]["distance"].item()

    restaurant_index = {}
    for key in restaurants:
        for value in restaurant:
            restaurant_index[key] = value
            restaurant.remove(value)
            break  

    customer_index = {}
    for key in customers:
        for value in customer:
            customer_index[key] = value
            customer.remove(value)
            break  


    location_distance = {}
    for i in start:
        for j in end: 
            if loc_index[i] == loc_index[j]:
                location_distance[(i,j)] = 0 
            else:
                location_distance[(i,j)] = df_distance[(df_distance["origin"] == loc_index[i]) \
                                                      &(df_distance["destination"] == loc_index[j])]["distance"].values


    min_arrival_time_mins = {}
    for j in end:
        if j in restaurants:
            delta = pd.to_datetime(df.loc[df["restaurant"] == loc_index[j]]["estimated availability"].tolist()[0]) - start_time
            min_arrival_time_mins[j] = delta/datetime.timedelta(minutes=1)
        if j in customers: 
            min_arrival_time_mins[j] = 0 

    drivers_start = {}
    for v in vehicle:
        drivers_start[v] = df_drivers4.iloc[v][0]

    drivers_velocity = {}
    for v in vehicle:
        drivers_velocity[v] = 60 /df_drivers4.iloc[v][1] # convert to minutes / km 


    #Calculate travel time for each distance between i and j
    wait_time = 5
    travel_time = {}
    for i in start:
        for j in end:
            for v in vehicle:
                if i == j:
                    travel_time[(i,j,v)] = 0 
                else: 
                    if i in customers:
                        travel_time[(i,j,v)] = location_distance[(i,j)]*drivers_velocity[v] + wait_time
                    else:
                        travel_time[(i,j,v)] = location_distance[(i,j)]*drivers_velocity[v] 

    start_region = []
    for v in vehicle:
        start_region.append(df_drivers4["start region"][v])
        
        
    # == Variables ==    
    xvar = pulp.LpVariable.dict("x", (start, end, vehicle, step), cat=pulp.LpBinary)
    dvar = pulp.LpVariable.dict("d", (vehicle, step), lowBound=0.0, cat=pulp.LpContinuous)
    wvar = pulp.LpVariable.dict("w", (end, vehicle, step), lowBound=0.0, cat=pulp.LpContinuous)
    zvar = pulp.LpVariable.dict("z", (vehicle, step), lowBound=0.0, cat=pulp.LpContinuous)
    
    
    # == Model and Constraints ==
    flowmodel = pulp.LpProblem(name = "FlowModel", sense = pulp.LpMinimize)

    # Constraint 1: every location visited once by one vehicle
    for j in end: 
        flowmodel += pulp.lpSum([xvar[(i,j,v,t)] for i in start for v in vehicle for t in step]) == 1

    # #Constraint 2: Force every vehicle takes at least one order:
    # for v in vehicle:
    #     flowmodel += pulp.lpSum([xvar[i,j,v,t]] for i in start for j in end for t in step) >= 1

    # Constraint 3: Conservation of flow
    for v in vehicle:  
        for t in step[:-1]:
            for j in end:
                flowmodel += (pulp.lpSum([xvar[(i,j,v,t)] for i in start]) == pulp.lpSum([xvar[(j,k,v,t+1)] for k in end]))

    # Constraint 4: First node of each driver has an outflow of 1 in the first step, others are zero
    for v in vehicle:
        flowmodel += pulp.lpSum(xvar[(v,j,v,t)] for j in end for t in step) == 1

    for i in end:
        for v in vehicle:
            flowmodel += pulp.lpSum(xvar[(i,j,v,0)] for j in end) == 0


    # Constraint 5: for each vehicle at each time step, there should be only 1 xvar <= 1 (=0 if nothing happened within this time block)
    for v in vehicle:
        for t in step:
            flowmodel += pulp.lpSum(xvar[(i,j,v,t)] for i in start for j in end) <= 1

    # Constraint 6: make sure driver goes to a restaurant before delivering the food
    def get_key_from_restaurant(val):
        for key, value in restaurant_index.items():
             if val == value:
                return key
            
    def get_val_from_restaurant_key(key):
        for key, value in restaurant_index.items():
             if key == key:
                return value        

    def get_key_from_customer(val):
        for key, value in customer_index.items():
             if val == value:
                return key
            
    def get_val_from_customer_key(key):
        for key, value in customer_index.items():
             if key == key:
                return value  

    def find_restaurant_of_customer(customer):
        for order in list_order:
            if customer == get_key_from_customer(order[1]):
                return get_key_from_restaurant(order[0])

    for t in step: 
        for v in vehicle:
            for j in end: 
                if j in customers: 
                    flowmodel += pulp.lpSum( [xvar[i,j,v,t] for i in start]) <= pulp.lpSum([xvar[i, find_restaurant_of_customer(j),v, t_] for i in start for t_ in step[:t]])

    # # Constraint 7: Time to arrive at the location of the t-th step
    for v in vehicle:
        for t in step:
            if t == 0: 
                flowmodel += dvar[v,t] >= pulp.lpSum([ xvar[(i,j,v,0)] * min_arrival_time_mins[j] for i in start for j in end])
                flowmodel += dvar[v,t] >= pulp.lpSum([ xvar[(i,j,v,t)] * travel_time[(i,j,v)] for i in start for j in end])
            else:
                flowmodel += dvar[v,t] >= (dvar[v,t-1] + pulp.lpSum([ xvar[(i,j,v,t)] * travel_time[(i,j,v)] for i in start for j in end]))      

    # # Constraint 8: if arriving at location j, ensures time is at least the min_arrival_time_mins
    for t in step:
        for i in start:
            for j in end:
                for v in vehicle:
                    flowmodel += dvar[v,t] >= min_arrival_time_mins[j] * xvar[i,j,v,t]


    # # Constraint 9: ensures the average waiting time of the orders is lower than the maximum W
    # implementation idea: 
    #         flowmodel += pulp.lpSum([wvar[j,t] * xvar[i,j,t] for i in start]) <= W
    # use zvar[t] with big Constant M to denote wvar[j,t] * xvar[i,j,t] as two decision variables cannot be multiplied together
    # zvar[t] is the total waiting time up until the step t
    # Constraint: define waiting time of each order at step t: wvar[j,t]

    for v in vehicle:
        for t in step:
            for j in end:
                if j in restaurants:
                    flowmodel += wvar[(j,v,t)]  == 0
                else:
                    flowmodel += wvar[(j,v,t)] == dvar[v,t] - pulp.lpSum([ min_arrival_time_mins[find_restaurant_of_customer(j)] * xvar[(i,j,v,t)] for i in start])


    M = 100000
    for t in step:
        for i in start:
            for j in end:
                for v in vehicle:
                    if t == 0:
                        flowmodel += zvar[v,t] == 0 
                    elif t == 1:
                        flowmodel += zvar[v,t] >= wvar[j,v,t] - M * (1 - xvar[(i,j,v,t)])
                        flowmodel += zvar[v,t] <= wvar[j,v,t] + M * (1 - xvar[(i,j,v,t)])
                    else: 
                        flowmodel += zvar[v,t] >= zvar[v,t-1] + wvar[(j,v,t)] - M * (1 - xvar[(i,j,v,t)])
                        flowmodel += zvar[v,t] <= zvar[v,t-1] + wvar[(j,v,t)] + M * (1 - xvar[(i,j,v,t)])


    for t in step:
        flowmodel += pulp.lpSum(zvar[v,t] for v in vehicle) <= W * len(customers)

        
    # == Solve ==    
    obj = pulp.lpSum([location_distance[(i,j)] * xvar[(i,j,v,t)] for i in start for j in end for v in vehicle for t in step])
    flowmodel += obj

    flowmodel.solve()
    print("Status:", pulp.LpStatus[flowmodel.status])

    # Total Distance
    totalDistance = pulp.value(flowmodel.objective)
    print("Total Distance: " + str(totalDistance))
    
    SumDis += totalDistance
    
    # # print solution
    for t in step: 
        print("Step: " + str(t))
        #print("Total travel time (in minutes) up to this step: " + str(dvar[t].varValue))

        for v in vehicle:
            for i in start: 
                for j in end: 

                    if xvar[(i,j,v,t)].varValue > 0.0:
                        print("\tVehicle: " + str(v))
                        print("\tLeave from " + str(i) + ": " + str(loc_index.get(i)))
                        print("\tTravel to " + str(j) + ": " + str(loc_index.get(j)))
                        print("\tWaiting time: " + str(wvar[(j,v,t)].varValue))
#                         print("\tTotal waiting time up to this step: " + str(zvar[v,t].varValue))


    total_waiting_time = 0
    for v in vehicle:
        for t in step:
            total_waiting_time += zvar[v,t].varValue
            
#     ClusterWait += zvar[t].varValue
#     ClusterCust += len(customers)

    loc_i=[]
    loc_j=[]
    for v in vehicle:
        for t in step: 
            for i in start:
                for j in end:
                    if xvar[(i,j,v,t)].varValue > 0.0:
                        loc_j.append(str(loc_index.get(i)))
                        loc_i.append(str(loc_index.get(i)))

    loc_i=pd.DataFrame(loc_i, columns=['loc_i'])
    loc_j=pd.DataFrame(loc_j, columns=['loc_j'])
    steps=pd.DataFrame(step, columns=['steps'])

    dfviz = pd.concat([loc_i, loc_j],axis=1, join='inner')
    dfviz = pd.concat([dfviz, steps], axis=1, join='inner')
    
    #new1 = dfviz['loc_i'].str.split('_')
    #dfviz['Tempi'] = [item[0] for item in new1]

    new2 = dfviz['loc_j'].str.split('_')
    dfviz['Tempj'] = [item[0] for item in new2]

    new2 = dfviz['loc_i'].str.split('_')
    dfviz['Tempi'] = [item[0] for item in new2]

    dfviz2 = dfviz.merge(regions[['name', 'latitude', 'longitude']], how = 'left',
                left_on = 'Tempi', right_on = 'name').drop(columns = ['name'])

    dfviz2=dfviz2.rename(columns={"latitude": "lat_i", "longitude": "long_i"})

    dfviz2 = dfviz2.merge(regions[['name', 'latitude', 'longitude']], how = 'left',
                left_on = 'Tempj', right_on = 'name').drop(columns = ['name'])

    dfviz2=dfviz2.rename(columns={"latitude": "lat_j", "longitude": "long_j"})

    #m= folium.Map(location=[43.6532,  -79.3832])
    marker_icon='cloud'
    popup='Default popup Marker1'

    #Setting location markers for locations i
    for i in range(0, len(dfviz2.index)):
        folium.Marker(location=[dfviz2['lat_i'][i],  dfviz2['long_i'][i]],tooltip=dfviz2['loc_i'][i],icon=folium.Icon(color="green")).add_to(m)

    #Setting location markers for locations j
    for i in range(0, len(dfviz2.index)):
        folium.Marker(location=[dfviz2['lat_j'][i],  dfviz2['long_j'][i]],tooltip=dfviz2['loc_j'][i],icon=folium.Icon(color="red")).add_to(m)

    route = []

    route.append([dfviz2['lat_i'][0], dfviz2['long_i'][0]])

    for i in range(0, len(dfviz2.index)):
        route.append([dfviz2['lat_j'][i], dfviz2['long_j'][i]])

    #Defining a feature group

    string = "Route" + str(cluster)

    f1=folium.FeatureGroup("string")



    line_1=folium.vector_layers.PolyLine(route,popup='<b>Path of Vehicle_1</b>',sticky=True,tooltip='Route 1',color=colors[cluster],weight=3).add_to(f1)

    for i in range(0, len(route)-1):
        loc1=route[i]
        loc2=route[i+1]
        arrows = getArrows(locations=[loc1,loc2], n_arrows=3)
        for arrow in arrows:
            arrow.add_to(m)

    f1.add_to(m)
    maps.append(m)

[0, 1]
Status: Optimal
Total Distance: 31.316162535976254
Step: 0
Step: 1
Step: 2
Step: 3
Step: 4
Step: 5
Step: 6
Step: 7
Step: 8
	Vehicle: 0
	Leave from 0: Downtown Toronto (Richmond / Adelaide / King)
	Travel to 5: Downtown Toronto (Underground city)
	Waiting time: 0.0
	Vehicle: 1
	Leave from 1: Downtown Toronto (St. James Park)
	Travel to 3: Downtown Toronto (Ryerson)
	Waiting time: 0.0
Step: 9
	Vehicle: 0
	Leave from 5: Downtown Toronto (Underground city)
	Travel to 2: Downtown Toronto (Central Bay Street)
	Waiting time: 0.0
	Vehicle: 1
	Leave from 3: Downtown Toronto (Ryerson)
	Travel to 4: Downtown Toronto (St. James Town / Cabbagetown)
	Waiting time: 0.0
Step: 10
	Vehicle: 0
	Leave from 2: Downtown Toronto (Central Bay Street)
	Travel to 6: Downtown Toronto (University of Toronto / Harbord)
	Waiting time: 0.0
	Vehicle: 1
	Leave from 4: Downtown Toronto (St. James Town / Cabbagetown)
	Travel to 7: Downtown Toronto (St. James Park)_1
	Waiting time: 0.0
Step: 11
	Vehicle: 0
	Leave 

In [ ]:
SumDis

44.23722785087357

In [ ]:
m

In [ ]:
        # print(total_waiting_time/len(customers))
# # print("Average waiting time: " + str(zvar[v,t].varValue/len(customers)))